**Neural network experiment**

# **import user-specified packages and google drive files**

In [ ]:
%xmode Verbose

Exception reporting mode: Verbose


In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import dataset_confs
from DatasetManager import DatasetManager

In [2]:
csv_files = {
    #"bpic2011": ["BPIC11_f%s"%formula for formula in range(2,3)],
    "bpic2015": ["BPIC15_%s_f2"%(municipality) for municipality in range(2,3)],
    #"sepsis_cases": ["sepsis_cases_2","sepsis_cases_4"],
    #"bpic2012": ["bpic2012_O_ACCEPTED-COMPLETE"],
    #production": ["Production"],
    #"bpic2017": ["BPIC17_O_Accepted","BPIC17_O_Cancelled","BPIC17_0_Refused"],
    #"traffic_fines": ["traffic_fines_%s"%formula for formula in range(1,2)],
    #"hospital_billing": ["hospital_billing_%s"%suffix for suffix in [2,3]]
}
files = []
for k, v in csv_files.items():
    files.extend(v)
dataset_ref_to_datasets = {
    # "bpic2011": ["bpic2011_f%s"%formula for formula in range(2,3)],
    "bpic2015": ["bpic2015_%s_f2"%(municipality) for municipality in range(2,3)],
    #"sepsis_cases": ["sepsis_cases_2","sepsis_cases_4"],
    #"bpic2012": ["bpic2012_accepted"],
    #"production": ["production"],
    #"bpic2017": ["bpic2017_accepted","bpic2017_cancelled","bpic2017_refused"],
    #"traffic_fines": ["traffic_fines_%s"%formula for formula in range(1,2)],
    #"hospital_billing": ["hospital_billing_%s"%suffix for suffix in [2,3]]
}

files = []
for k, v in csv_files.items():
    files.extend(v)
datasets = []
for k, v in dataset_ref_to_datasets.items():
    datasets.extend(v)
res = {datasets[i]: files[i] for i in range(len(datasets))}

In [4]:
print(datasets,res)

['bpic2015_2_f2'] {'bpic2015_2_f2': 'BPIC15_2_f2'}


# **import packages and functions**

In [5]:
#functions and packages
import pandas as pd
import numpy as np
import os
import pickle
from sklearn.metrics import roc_auc_score
from sklearn.base import BaseEstimator, TransformerMixin
from pandas.api.types import is_string_dtype
from collections import OrderedDict

#LSTM
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Nadam, Adam, SGD, RMSprop
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import tensorflow.keras.utils as ku
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import layers

#hyperopt
import hyperopt
from hyperopt import hp, Trials, fmin, tpe, STATUS_OK
from hyperopt.pyll.base import scope


#package from https://github.com/irhete/predictive-monitoring-benchmark/blob/master/experiments/experiments.py
from DatasetManager import DatasetManager

# **Own created functions**

In [6]:

#functions
#https://towardsdatascience.com/using-neural-networks-with-embedding-layers-to-encode-high-cardinality-categorical-variables-c1b872033ba2
class ColumnEncoder(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.columns = None
        self.maps = dict()

    def transform(self, X):
        X_copy = X.copy()
        for col in self.columns:
            # encode value x of col via dict entry self.maps[col][x]+1 if present, otherwise 0
            X_copy.loc[:,col] = X_copy.loc[:,col].apply(lambda x: self.maps[col].get(x, -1)+1)
        return X_copy

    def inverse_transform(self, X):
        X_copy = X.copy()
        for col in self.columns:
            values = list(self.maps[col].keys())
            # find value in ordered list and map out of range values to None
            X_copy.loc[:,col] = [values[i-1] if 0<i<=len(values) else None for i in X_copy[col]]
        return X_copy

    def fit(self, X, y=None):
        # only apply to string type columns
        self.columns = [col for col in X.columns if is_string_dtype(X[col])]
        for col in self.columns:
            self.maps[col] = OrderedDict({value: num for num, value in enumerate(sorted(set(X[col])))})
        return self

def prepare_inputs(X_train, X_test, data):  
    global ce
    ce = ColumnEncoder()
    X_train, X_test = X_train.astype(str), X_test.astype(str)
    X_train_enc = ce.fit_transform(X_train)
    X_test_enc = ce.transform(X_test)
    return X_train_enc, X_test_enc
   
def create_index(log_df, column):
    """Creates an idx for a categorical attribute.
    Args:
        log_df: dataframe.
        column: column name.
    Returns:
        index of a categorical attribute pairs.
    """
    temp_list = temp_list = log_df[log_df[column] != 'none'][[column]].values.tolist() #remove all 'none' values from the index
    subsec_set = {(x[0]) for x in temp_list}
    subsec_set = sorted(list(subsec_set))
    alias = dict()
    if column !='next_activity':
      for i, _ in enumerate(subsec_set):          
          alias[subsec_set[i]] = i + 1
      alias['none'] = 0
    else:
      for i, _ in enumerate(subsec_set):
          alias[subsec_set[i]] = i  
    #reorder by the index value
    alias = {k: v for k, v in sorted(alias.items(), key=lambda item: item[1])}
    return alias

def create_indexes(i, data):
    dyn_index = create_index(data, i)
    index_dyn = {v: k for k, v in dyn_index.items()}
    dyn_weights = ku.to_categorical(sorted(index_dyn.keys()), len(dyn_index))
    return dyn_weights,  dyn_index, index_dyn

def normalize_events(log_df,features):
    """[summary]

    Args:
        log_df (DataFrame): The dataframe with eventlog data
        args (Dictionary): The set of parameters
        features (list): the list of feature name

    Returns:
        Dataframe: Returns a Dataframe with normalized numerical features
    """
    for feature in features:
            min_feature= np.min(log_df[feature])
            max_feature= np.max(log_df[feature])
            if max_feature ==0 or max_feature-min_feature==0:
                max_feature+=0.00001
            minmax = lambda x: (2*(x[feature]-min_feature)/(max_feature-min_feature))-1
            log_df['%s_norm'%(feature)] = log_df.apply(minmax, axis=1)
    return log_df

def groupby_caseID(data, cols):
    ans = [pd.DataFrame(y) for x, y in data[cols].groupby('Case ID', as_index=False)]
    return ans

def remove_punctuations(columns_before):
    columns = []
    for string in columns_before:
        new_string = string.replace(":", "_")
        columns.append(new_string)
    return columns

def labels_after_grouping(data_train,data_test):
    train_labels = []
    for i in range (0,len(data_train)):
        temp_label = data_train[i]['label'].iloc[0]
        train_labels.append(temp_label)

    test_labels = []
    for i in range (0,len(data_test)):
        temp_label = data_test[i]['label'].iloc[0]
        test_labels.append(temp_label)
    train_y = [1 if i!='regular' else 0 for i in train_labels]
    test_y = [1 if i!='regular' else 0 for i in test_labels]
    return train_y, test_y

def pad_data(cols, data, maxlen):
    #padding of the different categorical columns
    #train paddings
    paddings = []
    for i in cols:
        padding= []
        for k in range(0,len(data)):
            temp = []
            temp = list(data[k][i])
            padding.append(temp)
        padded = np.array(pad_sequences(padding,maxlen=maxlen, padding='pre', truncating='pre',value=0))
        paddings.append(padded)
    return paddings

def reshape_num_data(pad_data, cutoff):
        pad_num = np.reshape(pad_data, (len(pad_data), cutoff, 1))
        return pad_num
    
def processString(txt):
    specialChars = "():. " 
    for specialChar in specialChars:
      txt = txt.replace(specialChar, '')
    return txt

# **parameters**

In [7]:
# parameters
params_dir = './content/drive/MyDrive/CurrentWork/Robustness/params_dir_DL'
column_selection= 'all'
cls_encoding ='OHE'
classifiers =['LSTM']
n_iter = 1
n_splits = 3
train_ratio = 0.8
random_state = 22

allow_negative=False
incl_time = True 
incl_res = True
# create results directory
if not os.path.exists(os.path.join(params_dir)):
    os.makedirs(os.path.join(params_dir))

# **Create data**

In [8]:
# function for preprocessing data

def create_data(dt_train_prefixes, dt_test_prefixes):
    #get the label of the train and test set
    test_y = dataset_manager.get_label_numeric(dt_test_prefixes)
    train_y = dataset_manager.get_label_numeric(dt_train_prefixes)

    #cat columns integerencoded
    dt_train_prefixes[cat_cols],dt_test_prefixes[cat_cols]= prepare_inputs(dt_train_prefixes[cat_cols], dt_test_prefixes[cat_cols], data)
    dt_train_prefixes[cat_cols] = dt_train_prefixes[cat_cols]+1
    dt_test_prefixes[cat_cols] = dt_test_prefixes[cat_cols]+1
   
    #[-1,1] normalize the numerical columns
    dt_train_prefixes = normalize_events(dt_train_prefixes,num_cols)
    dt_test_prefixes = normalize_events(dt_test_prefixes,num_cols)

    #groupby case ID
    ans_train = groupby_caseID(dt_train_prefixes, label_case_cols)
    ans_test = groupby_caseID(dt_test_prefixes, label_case_cols)
    #obtain the new label lists after grouping
    train_y, test_y = labels_after_grouping(ans_train, ans_test)
   
    ######DYNAMIC_COLS########
    #activity
    activity_train = pad_data(activity_col, ans_train, maxlen)
    activity_test = pad_data(activity_col, ans_test, maxlen)
    
    #pad dynamic cat columns
    paddings_train = pad_data(dynamic_cat_cols, ans_train, maxlen)
    paddings_test = pad_data(dynamic_cat_cols, ans_test, maxlen)
    
    #pad dynamic num columns
    paddings_train2 = pad_data(dynamic_num_cols, ans_train, maxlen)
    paddings_test2 = pad_data(dynamic_num_cols, ans_test, maxlen)
  
    #STATIC COLUMNS
    #static_num_cols 
    pad_train =pad_data(static_num_cols, ans_train, maxlen)
    pad_test = pad_data(static_num_cols, ans_test, maxlen)
    
    #static_cat_cols
    pad_train2 =pad_data(static_cat_cols, ans_train, maxlen)
    pad_test2 = pad_data(static_cat_cols, ans_test, maxlen)
    
    return pad_train, pad_test, pad_train2, pad_test2, paddings_train, paddings_test, paddings_train2,paddings_test2, activity_train, activity_test, train_y, test_y

# **Create and evaluate model**

In [9]:
def create_and_evaluate_model(args):  
    global trial_nr
    trial_nr += 1
    score = 0
    for cv_iter in range(n_splits):
          dt_test_prefixes_original = dt_prefixes[cv_iter]
          dt_train_prefixes_original = pd.DataFrame()
          for cv_train_iter in range(n_splits): 
              if cv_train_iter != cv_iter:
                  dt_train_prefixes_original = pd.concat([dt_train_prefixes_original, dt_prefixes[cv_train_iter]], axis=0)
    preds_all = []
    test_y_all = []
    dt_train_prefixes = dt_test_prefixes_original.copy()
    dt_test_prefixes = dt_test_prefixes_original.copy()
    pad_train, pad_test, pad_train2, pad_test2, paddings_train, paddings_test,paddings_train2,paddings_test2, activity_train, activity_test, train_y, test_y = create_data(dt_train_prefixes, dt_test_prefixes)
    print(args) 
    LSTMlayers = []
    input_layers = []
    LSTMlayers = []     
    alpha = layers.Bidirectional(layers.LSTM(args['lstm_size_alpha'], return_sequences=True,dropout= 0.3), name='alpha')
    alpha_dense = layers.Dense(1, kernel_regularizer=l2(0.001))
    model_inputs = []
    model_inputs_test = []
    
    for i in range(0,len(activity_col)):
        dyn_weights, dyn_index, index_dyn = create_indexes(activity_col[i], data)
        input_ohe = to_categorical(activity_train[i], num_classes=len(dyn_index)+1)
        #remove mask
        input_ohe = input_ohe[:, :, 1:]
        model_inputs.append(input_ohe)
        input_ohe_test = to_categorical(activity_test[i], num_classes=len(dyn_index)+1)
        input_ohe_test = input_ohe_test[:, :, 1:]
        model_inputs_test.append(input_ohe_test)
        dyn_cat_name =processString(activity_col[i])
        input_layer = layers.Input(shape=(input_ohe.shape[1],len(dyn_index), ), name=dyn_cat_name)
        input_layers.append(input_layer)
        #Set up the LSTM networks
        beta = layers.Bidirectional(layers.LSTM(args['lstm_size_beta'], dropout=0.3,return_sequences=True), name='beta'+str(dyn_cat_name))
        beta_out = beta(input_layer)
        #Dense layer for attention
        betadense = layers.Dense(dyn_weights.shape[1],activation='tanh', kernel_regularizer=l2(0.001))
        beta_out = layers.TimeDistributed(betadense, name='feature_attention_'+dyn_cat_name)(beta_out)
        c_t = layers.Multiply(name = dyn_cat_name+'_importance')([beta_out,input_layer])
        LSTMlayers.append(c_t)
    
    for i in range(0,len(dynamic_cat_cols)):
            dyn_weights, dyn_index, index_dyn = create_indexes(dynamic_cat_cols[i], data)
            input_ohe = to_categorical(paddings_train[i], num_classes=len(dyn_index)+1)
            input_ohe = input_ohe[:, :, 1:]
            input_ohe_test = to_categorical(paddings_test[i], num_classes=len(dyn_index)+1)
            input_ohe_test = input_ohe_test[:, :, 1:]
            model_inputs.append(input_ohe)
            model_inputs_test.append(input_ohe_test)
            dyn_cat_name =processString(dynamic_cat_cols[i])
            input_layer = layers.Input(shape=(input_ohe.shape[1],len(dyn_index), ), name=dyn_cat_name)
            input_layers.append(input_layer)
            #Set up the LSTM networks
            beta = layers.Bidirectional(layers.LSTM(args['lstm_size_beta'], dropout=0.3, return_sequences=True),
                                            name='beta'+str(dyn_cat_name))
            beta_out = beta(input_layer)
            #Dense layer for attention
            betadense = layers.Dense(dyn_weights.shape[1],
                                        activation='tanh', kernel_regularizer=l2(0.001))

            beta_out = layers.TimeDistributed(betadense, name='feature_attention_'+dyn_cat_name)(beta_out)
            c_t = layers.Multiply(name = dyn_cat_name+'_importance')([beta_out,input_layer])
            LSTMlayers.append(c_t)

    for i in range(0,len(dynamic_num_cols)):
            dyn_num_name =processString(dynamic_num_cols[i])
            input_layer = layers.Input(shape=(cutoff,1, ), name=dyn_num_name)
            model_inputs.append(paddings_train2[i])
            model_inputs_test.append(paddings_test2[i])
            input_layers.append(input_layer)
            #Set up the LSTM networks
            beta = layers.Bidirectional(layers.LSTM(args['lstm_size_beta'],dropout=0.3, return_sequences=True), name='beta'+str(dyn_num_name))
            beta_out = beta(input_layer)
            #Dense layer for attention
            betadense = layers.Dense(1, activation='tanh', kernel_regularizer=l2(0.001))

            beta_out = layers.TimeDistributed(betadense, name='feature_attention_'+dyn_num_name)(beta_out)
            c_t = layers.Multiply(name = dyn_num_name+'_importance')([beta_out,input_layer])
            LSTMlayers.append(c_t)
    
    for i in range(0,len(static_num_cols)):
            static_num_name =processString(static_num_cols[i])
            input_layer = layers.Input(shape=(cutoff,1), name=static_num_name)
            input_layers.append(input_layer)
            betadense = layers.Dense(1, activation='tanh', kernel_regularizer=l2(0.001))
            beta_out = layers.TimeDistributed(betadense, name='feature_attention_'+static_num_name)(input_layer)
            c_t = layers.Multiply(name = static_num_name+'_importance')([beta_out,input_layer])
            LSTMlayers.append(c_t)

            model_inputs.append(reshape_num_data(pad_train[i], cutoff))
            model_inputs_test.append(reshape_num_data(pad_test[i], cutoff))

    for i in range(0,len(static_cat_cols)):
           stat_weights, stat_index, index_stat = create_indexes(static_cat_cols[i], data)
           input_ohe = to_categorical(pad_train2[i], num_classes=len(stat_index)+1)
           input_ohe = input_ohe[:, :, 1:]
           input_ohe_test = to_categorical(pad_test2[i], num_classes=len(stat_index)+1)
           model_inputs.append(input_ohe)
           input_ohe_test = input_ohe_test[:, :, 1:]
           model_inputs_test.append(input_ohe_test)
           stat_cat_name =processString(static_cat_cols[i])
           input_layer = layers.Input(shape=(input_ohe.shape[1],len(stat_index), ), name=stat_cat_name)
           input_layers.append(input_layer)
           betadense = layers.Dense(1, activation='tanh', kernel_regularizer=l2(0.001))
           beta_out = layers.TimeDistributed(betadense, name='feature_attention_'+stat_cat_name)(input_layer)
           c_t = layers.Multiply(name = stat_cat_name+'_importance')([beta_out,input_layer])
           LSTMlayers.append(c_t)
           
    total_layers = LSTMlayers
    c_t = layers.concatenate(total_layers,name = 'concat')
    #Compute alpha, timestep attention
    alpha_out = alpha(c_t)
    alpha_out = layers.TimeDistributed(alpha_dense, name='alpha_dense')(alpha_out)
    alpha_out = layers.Softmax(name='timestep_attention', axis=1)(alpha_out)

    #Compute context vector based on attentions and embeddings
    c_t = layers.Multiply()([alpha_out, c_t])
    c_t = layers.Lambda(lambda x: backend.sum(x, axis=1))(c_t)

    #contexts = L.concatenate([c_t,age_input,cl_input_d], name='contexts')
    contexts = layers.Dropout(0.3)(c_t)

    output_layer = Dense(1, activation='sigmoid', name='final_output')(contexts)
  
    #MODEL     
    model = Model(inputs=input_layers, outputs=output_layer)
    model.compile(loss={'final_output':'binary_crossentropy'}, optimizer= Nadam(learning_rate= 0.0001))

    model.summary()
           
    early_stopping = EarlyStopping(monitor='val_loss', mode='auto', patience=5,min_delta=0.001)
    lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.8, patience=3, verbose=2, mode='auto')
            
    result = model.fit(model_inputs,
              np.array(train_y),
              callbacks=[early_stopping, lr_reducer],
              validation_split = 0.1,
              verbose=2, batch_size=args['batch_size'],
              epochs=100)
            
    # Get the lowest validation loss of the training epochs
    validation_loss = np.amin(result.history['val_loss']) 
    print('Best validation loss of epoch:', validation_loss)
        
    pred = model.predict(model_inputs_test)
    pred3  =[pred[i][0] for i in range(0,len(pred))]
    #pred2 = pred.flatten()
    preds_all.extend(pred3)
    test_y_all.extend(test_y)
    score += roc_auc_score(test_y_all, preds_all)
    for k, v in args.items():
          fout_all.write("%s;%s;%s;%s;%s;%s;%s\n" % (trial_nr, dataset_name, cls_method, method_name, k, v, score / n_splits))  
         
    fout_all.write("%s;%s;%s;%s;%s\n" % (trial_nr, dataset_name, cls_method, method_name, 0))   
 
    fout_all.flush()
    return {'loss': validation_loss, 
            'status': STATUS_OK, 
            'model': model, 
            'args': args}

# **loop over datasets and classifiers**

In [12]:
# Model
for cls_method in classifiers:
    for dataset_name in datasets:
        print('Dataset:', dataset_name)
        print('Classifier', cls_method)
        print('Encoding', cls_encoding)
        method_name = "%s_%s"%(column_selection, cls_encoding)            

        # read the data
        dataset_manager = DatasetManager(dataset_name)
        data = dataset_manager.read_dataset()   
     
        cls_encoder_args = {'case_id_col': dataset_manager.case_id_col, 
                        'static_cat_cols': dataset_manager.static_cat_cols,
                        'static_num_cols': dataset_manager.static_num_cols, 
                        'dynamic_cat_cols': dataset_manager.dynamic_cat_cols,
                        'dynamic_num_cols': dataset_manager.dynamic_num_cols, 
                        'fillna': True}

        #save columns
        dynamic_cat_cols = cls_encoder_args['dynamic_cat_cols'].copy()
        #ACTIVITY COL
        activity_col =  [x for x in dynamic_cat_cols if 'Activity' in x]
        dynamic_cat_cols.remove(activity_col[0])
        dynamic_num_cols = cls_encoder_args['dynamic_num_cols'].copy()
        dynamic_num_cols.remove('event_nr')
        static_cat_cols = cls_encoder_args['static_cat_cols'].copy()
        static_num_cols = cls_encoder_args['static_num_cols'].copy()
        #COLUMNS
        cat_cols = dynamic_cat_cols + static_cat_cols +activity_col
        num_cols = dynamic_num_cols + static_num_cols
        dynamic_cols = dynamic_cat_cols+ dynamic_num_cols + activity_col
        static_cols = static_cat_cols+ static_num_cols
        
        #TOTAL COLS
        cols =  cat_cols + num_cols
        
        #groupby data per Case ID and extract label
        label_case_cols = cols + [cls_encoder_args['case_id_col']] + ['label']
        
        #file to save results
        outfile = os.path.join(params_dir, "performance_results_%s_%s_%s.csv" % (cls_method, dataset_name, method_name))
            
        # determine min and max (truncated) prefix lengths
        min_prefix_length = 1
        if "traffic_fines" in dataset_name:
            max_prefix_length = 10
        elif "bpic2017" in dataset_name:
            max_prefix_length = min(20, dataset_manager.get_pos_case_length_quantile(data, 0.90))
        else:
            max_prefix_length = min(40, dataset_manager.get_pos_case_length_quantile(data, 0.90))
        maxlen = cutoff = max_prefix_length

        # split into training and test
        train, _ = dataset_manager.split_data_strict(data, train_ratio, split="temporal")
    
        # prepare chunks for CV
        dt_prefixes = []
        class_ratios = []
        for train_chunk, test_chunk in dataset_manager.get_stratified_split_generator(train, n_splits=n_splits):
                class_ratios.append(dataset_manager.get_class_ratio(train_chunk))
                # generate data where each prefix is a separate instance
                dt_prefixes.append(dataset_manager.generate_prefix_data(test_chunk, min_prefix_length, max_prefix_length))
        del train
        # set up search space
        if cls_method == "LSTM":
                space = {'lstm_size_alpha' :      scope.int(hp.quniform('lstm_size_alpha',8,16,8)),
                         'lstm_size_beta' :       scope.int(hp.quniform('lstm_size_beta',8,16,8)),         
                         'batch_size' :           scope.int(hp.quniform('batch_size',16,64,16))}
  
        # optimize parameters
        trial_nr = 0
        trials = Trials()
        fout_all = open(os.path.join(params_dir, "param_optim_all_trials_%s_%s_%s.csv" % (cls_method, dataset_name, method_name)), "w")
        fout_all.write("%s;%s;%s;%s;%s;%s;%s\n" % ("iter", "dataset", "cls", "method", "param", "value", "score"))   
        best = fmin(create_and_evaluate_model, space, algo=tpe.suggest, max_evals=6, trials=trials)
        fout_all.close()
        
        # write the best parameters
        best_params = hyperopt.space_eval(space, best)
        outfile = os.path.join(params_dir, "optimal_params_%s_%s_%s.pickle" % (cls_method, dataset_name, method_name))
        # write to file
        with open(outfile, "wb") as fout:
            pickle.dump(best_params, fout)

Dataset: bpic2015_2_f2
Classifier LSTM
Encoding OHE
{'batch_size': 48, 'lstm_size_alpha': 16, 'lstm_size_beta': 16}
Model: "model"

__________________________________________________________________________________________________

 Layer (type)                   Output Shape         Param #     Connected to                     


 Activity (InputLayer)          [(None, 40, 397)]    0           []                               

 monitoringResource (InputLayer  [(None, 40, 10)]    0           []                               

 )                                                                                                

 question (InputLayer)          [(None, 40, 14)]     0           []                               

 orgresource (InputLayer)       [(None, 40, 12)]     0           []                               

 hour (InputLayer)              [(None, 40, 1)]      0           []                               

 weekday (InputLayer)           [(None, 40, 1)]      0           []

In [ ]:
print('hello')

hello
